In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Input & Output State

In [2]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

model = ChatOpenAI()

In [4]:
from typing import TypedDict


class ChatMessages(TypedDict):
    question: str
    answer: str
    llm_calls: int

In [5]:
def call_model(state: ChatMessages):
    question = state["question"]
    llm_calls = state.get("llm_calls", 0)
    state["llm_calls"] = llm_calls + 1
    print("LLM_CALLS:", state["llm_calls"])
    response = model.invoke(input=question)
    state["answer"] = response.content
    return state

In [6]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [7]:
graph.invoke(input={"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 1


{'question': 'Whats the highest mountain in the world?',
 'answer': 'The highest mountain in the world is Mount Everest, which stands at an elevation of 29,029 feet (8,848 meters) above sea level.',
 'llm_calls': 1}

In [8]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


class InputState(TypedDict):
    question: str


class PrivateState(TypedDict):
    llm_calls: int


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, PrivateState, OutputState):
    pass

In [9]:
workflow = StateGraph(OverallState, input=InputState, output=OutputState)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [10]:
graph.invoke({"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 1


{'answer': 'Mount Everest, located in the Himalayas on the border of Nepal and Tibet, is the highest mountain in the world with a peak elevation of 29,032 feet (8,848 meters) above sea level.'}

### Add runtime configuration

In [11]:
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.config import RunnableConfig
from langchain.schema import SystemMessage, HumanMessage


def call_model(state: OverallState, config: RunnableConfig):
    language = config["configurable"].get("language", "English")
    system_message_content = "Respond in {}".format(language)
    system_message = SystemMessage(content=system_message_content)
    messages = [system_message, HumanMessage(content=state["question"])]
    response = model.invoke(messages)
    return {"answer": response}

In [12]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [13]:
config = {"configurable": {"language": "Spanish"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

{'question': "What's the highest mountain in the world?",
 'answer': AIMessage(content='La montaña más alta del mundo es el Monte Everest, que se encuentra en la cordillera del Himalaya, en la frontera entre Nepal y China. Su pico alcanza una altitud de 8,848 metros sobre el nivel del mar.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 23, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c6da645e-8277-467c-abbf-99d9366aeee9-0', usage_metadata={'input_tokens': 23, 'output_tokens': 54, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}

In [15]:
config = {"configurable": {"language": "Tamil"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

{'question': "What's the highest mountain in the world?",
 'answer': AIMessage(content='உலகின் அதிசூடி மலை எது என்பது "எவெரெஸ்த்" மலை அல்லது "சக்ரா" மலை என்பது தான். இது நெபாளிய ஹிமாலயத்தில் அமைந்துள்ளது.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 23, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5fae9138-ad1a-469b-9775-2785aeca2743-0', usage_metadata={'input_tokens': 23, 'output_tokens': 159, 'total_tokens': 182, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}